# Figures

A notebook to make figures of mesh differences

In [1]:
import json
import pyvista as pv
import pymesh
from helpers.mesh import *
import cityjson

In [2]:
def load_citymodel(file):
    cm = json.load(file)

    if "transform" in cm:
        s = cm["transform"]["scale"]
        t = cm["transform"]["translate"]
        verts = [[v[0] * s[0] + t[0], v[1] * s[1] + t[1], v[2] * s[2] + t[2]]
                for v in cm["vertices"]]
    else:
        verts = cm["vertices"]

    # mesh points
    vertices = np.array(verts)

    return cm, vertices

def get_geometry(co, lod):
    """Returns the geometry of the given LoD.
    
    If lod is None then it returns the first one.
    """

    if len(co["geometry"]) == 0:
        return None

    if lod is None:
        return co["geometry"][0]

    for geom in co["geometry"]:
        if str(geom["lod"]) == str(lod):
            return geom

def is_valid(mesh):
    return mesh.volume > 0 and mesh.n_open_edges == 0

def get_mesh(objid, tile_id, lod="2.2"):
    filename = f"/home/alabetski/3DBAG_09/{tile_id}.json"

    with open(filename, "rb") as file:
        cm, verts = load_citymodel(file)

    building = cm["CityObjects"][objid]

    lod_dest = lod

    geom_dest = get_geometry(building, lod_dest)

    mesh_dest = cityjson.to_triangulated_polydata(geom_dest, verts)
    
    return geom_dest, mesh_dest, verts

In [1]:
import pymesh
import cityjson
import geometry

solid_color = "lightgrey"
trans_color = "red"
opacity = 0.7
edges_on = True
border_width = 5.0

def plot(geom, mesh, path, verts):
    pm_mesh = to_pymesh(mesh)

    pm_ch = pymesh.convex_hull(pm_mesh)

    convex_hull = to_pyvista(pm_ch)

    # Compute OBB with shapely
    obb_2d = cityjson.to_shapely(geom, verts, ground_only=False).minimum_rotated_rectangle
    min_z = np.min(mesh.clean().points[:, 2])
    max_z = np.max(mesh.clean().points[:, 2])
    obb = geometry.extrude(obb_2d, min_z, max_z)

    points = np.array([[p[0], p[1], min_z] for p in obb_2d.boundary.coords])
    obb.points += np.mean(points, axis=0)

    aobb = mesh.outline(generate_faces=True)

    p = pv.Plotter(window_size=[2048, 2048], shape=(2,2))
    p.add_title("test", color="black")

    p.background_color = "white"

    p.add_mesh(mesh, color=solid_color)
    if edges_on:
        p.add_mesh(mesh.extract_feature_edges(), color="black", line_width=border_width)

    p.subplot(0, 1)

    p.add_mesh(mesh, color=solid_color)
    p.add_mesh(convex_hull, color=trans_color, opacity=opacity)
    if edges_on:
        p.add_mesh(convex_hull.extract_feature_edges(feature_angle=10), color=trans_color, line_width=border_width)

    p.subplot(1, 0)

    p.add_mesh(mesh, color=solid_color)
    p.add_mesh(obb, color=trans_color, opacity=opacity)
    if edges_on:
        p.add_mesh(obb.extract_feature_edges(), color=trans_color, line_width=border_width)

    p.subplot(1, 1)

    p.add_mesh(mesh, color=solid_color)
    p.add_mesh(aobb, color=trans_color, opacity=opacity)
    if edges_on:
        p.add_mesh(aobb.extract_feature_edges(), color=trans_color, line_width=border_width)

    p.show()
    p.link_views()
    # p.set_position(np.mean(mesh.points, axis=0) + [50, 0, 30])
    p.save_graphic(path)
    p.close()

In [4]:
import pandas as pd
df = pd.read_csv("/home/alabetski/3DBAG_09/all/lod2.2.csv")

clean = df[df["valid"]].set_index("id")

In [6]:
import os

def get_bag_id(attribute, value):
    return buildings[attribute].sub(value).abs().idxmin()

buildings = clean[clean["ground_area"] > 20]
root_path = "/home/alabetski/figures"

for attribute in clean.columns:
    if attribute.endswith("_3d"):
        path = f"{root_path}/{attribute}"
        if not os.path.exists(path):
            os.mkdir(path)
        cols = [attribute, "tile_id", "ground_area"]
        
        for val in [0, 1] + [buildings[attribute].quantile(v) for v in np.arange(0, 1.01, 0.05)]:
            bag_id = get_bag_id(attribute, val)
            tile_id = buildings.loc[bag_id]["tile_id"]

            geom, mesh, verts = get_mesh(bag_id, tile_id)

            fig_name = f"{buildings.loc[bag_id][attribute]}_{bag_id}.pdf"
            fig_path = f"{path}/{fig_name}"
            plot(geom, mesh, fig_path, verts)
            print(f"{fig_path} done...")

ViewInteractiveWidget(height=2048, layout=Layout(height='auto', width='100%'), width=2048)

/home/alabetski/figures/hemisphericality_3d/0.0284824544070598_NL.IMBAG.Pand.0599100000657847-0.pdf done...


ViewInteractiveWidget(height=2048, layout=Layout(height='auto', width='100%'), width=2048)

/home/alabetski/figures/hemisphericality_3d/0.5952215940868545_NL.IMBAG.Pand.0363100012223144-0.pdf done...


ViewInteractiveWidget(height=2048, layout=Layout(height='auto', width='100%'), width=2048)

...

/home/alabetski/figures/roughness_index_3d/1.010698889000142_NL.IMBAG.Pand.0599100010002607-0.pdf done...


ViewInteractiveWidget(height=2048, layout=Layout(height='auto', width='100%'), width=2048)

/home/alabetski/figures/roughness_index_3d/1.1311893570347975_NL.IMBAG.Pand.1892100001079833-0.pdf done...


ViewInteractiveWidget(height=2048, layout=Layout(height='auto', width='100%'), width=2048)

/home/alabetski/figures/roughness_index_3d/61.25603908891532_NL.IMBAG.Pand.0344100000086977-0.pdf done...


In [7]:
np.arange(0, 1.1, 0.25)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])